# 2.1.3 Zeichenerklärung – OCR (GPT-4o)

* Zero-Shot, Chain-Of-Thought
* Beste Ergebnisse (img_prompts): B2

In [2]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
samantha = OpenAI(instructions)
parser = Parser()
runner = Runner()

### A) Schema Nutzungsschablone

In [3]:
# SKIP – Analog zu den anderen Fällen
msg213_bp872_nz = '''
Hier ist das Schema der Tabellenerklärung in ASCII-Tabellenform:

+----------------+------------------+
| Art der baulichen Nutzung         |
+----------------+------------------+
| Grundflächenzahl| Geschossflächen-|
| (GRZ)          | zahl (GF)       |
+----------------+------------------+
'''
%store msg213_bp872_nz

Stored 'msg213_bp872_nz' (str)


### B) Zeichenerklärung der Nutzungsschablone

In [4]:
# B1) Zeichenerklärung – Inhalt zu den Hauptkategorien – NO CONTEXT
pdf_path = "../data/raw/bpläne/3_xplanung/BP_872A_ZE.pdf"

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

async def run():
    return await samantha.extractTextFromImages(instruction, [pdf_path], img_type="pdf")
await runner.async_consistency_check(run)

['Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan Nr. 872 A der Stadt Augsburg:

### Art der baulichen Nutzung
- **WA1**: Allgemeines Wohngebiet mit Nummerierung

### Maß der baulichen Nutzung
- **GF 5250**: Geschossfläche (GF) als Höchstmaß
- **0,35**: Grundflächenzahl (GRZ) als Höchstmaß
- **II**: Zahl der Vollgeschosse, zwingend
- **IV**: Zahl der Vollgeschosse als Höchstmaß
- **OK 12,5**: Oberkante (OK) baulicher Anlagen in Meter zwingend, bezogen auf einen Bezugspunkt gemäß textlicher Festsetzung
- **OK 12,5**: Oberkante (OK) baulicher Anlagen in Meter als Höchstmaß, bezogen auf einen Bezugspunkt gemäß textlicher Festsetzung

### Bauweise
- **g**: Geschlossene Bauweise

### Überbaubare Grundstücksfläche
- **Baulinie**: Markiert durch eine durchgezogene rote Linie
- **Baugrenze**: Markiert durch eine gestrichelte rote Linie']
#############################################
['Hier sind die extrahierten Informationen zu den angeforderten Theme

[['Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan Nr. 872 A der Stadt Augsburg:\n\n### Art der baulichen Nutzung\n- **WA1**: Allgemeines Wohngebiet mit Nummerierung\n\n### Maß der baulichen Nutzung\n- **GF 5250**: Geschossfläche (GF) als Höchstmaß\n- **0,35**: Grundflächenzahl (GRZ) als Höchstmaß\n- **II**: Zahl der Vollgeschosse, zwingend\n- **IV**: Zahl der Vollgeschosse als Höchstmaß\n- **OK 12,5**: Oberkante (OK) baulicher Anlagen in Meter zwingend, bezogen auf einen Bezugspunkt gemäß textlicher Festsetzung\n- **OK 12,5**: Oberkante (OK) baulicher Anlagen in Meter als Höchstmaß, bezogen auf einen Bezugspunkt gemäß textlicher Festsetzung\n\n### Bauweise\n- **g**: Geschlossene Bauweise\n\n### Überbaubare Grundstücksfläche\n- **Baulinie**: Markiert durch eine durchgezogene rote Linie\n- **Baugrenze**: Markiert durch eine gestrichelte rote Linie'],
 ['Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan N

In [5]:
# B2) Zeichenerklärung – Inhalt zu den Hauptkategorien – WITH CONTEXT
# Kontext: Schema, BauNVO
%store -r msgbaunvo msg213_bp872_nz
schema_prompts = parser.text2prompts([msg213_bp872_nz])
baunvo_prompts = parser.text2prompts([msgbaunvo])
contexts = list(map(lambda schema: [*baunvo_prompts,schema], schema_prompts))

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

async def run():
    return await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], contexts, img_type="pdf")
results = await runner.async_consistency_check(run)
msg213_bp872_ze = results[0]
%store msg213_bp872_ze

['### Art der baulichen Nutzung

- **WA1**: Allgemeines Wohngebiet mit Nummerierung

### Maß der baulichen Nutzung

- **GF 5250**: Geschossfläche (GF) als Höchstmaß
- **0,35**: Grundflächenzahl (GRZ) als Höchstmaß
- **IV**: Zahl der Vollgeschosse, zwingend
- **IV**: Zahl der Vollgeschosse als Höchstmaß
- **OK 12,5**: Oberkante (OK) baulicher Anlagen in Meter zwingend, bezogen auf einen Bezugspunkt gemäß textlicher Festsetzung
- **OK 12,5**: Oberkante (OK) baulicher Anlagen in Meter als Höchstmaß, bezogen auf einen Bezugspunkt gemäß textlicher Festsetzung

### Bauweise, überbaubare Grundstücksfläche

- **g**: Geschlossene Bauweise
- **Baulinie**: (Symbol: rote Linie)
- **Baugrenze**: (Symbol: blaue Linie)']
#############################################
['### Art der baulichen Nutzung

- **WA1**: Allgemeines Wohngebiet mit Nummerierung

### Maß der baulichen Nutzung

- **GF 5250**: Geschossfläche (GF) als Höchstmaß
- **0,35**: Grundflächenzahl (GRZ) als Höchstmaß
- **IV**: Zahl der Vollg